In [1]:
import numpy as np
import pandas as pd
import random
import nltk
import pickle
import warnings
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
import zipfile
with zipfile.ZipFile('archivee.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [6]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')

In [7]:
movie = movies.merge(credits, on='title')

In [8]:
movie.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [9]:
movies.shape

(4803, 20)

In [10]:
movie = movie[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [11]:
movie.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


# Text preprocessing functions

In [12]:
lemmatizer = WordNetLemmatizer()
def convert(text):
    return [i['name'] for i in eval(text)]

def convert3(text):
    return [i['name'] for i in eval(text)[:3]]

def fetch_director(text):
    return [i['name'] for i in eval(text) if i['job'] == 'Director']


In [13]:
movie['genres'] = movie['genres'].apply(convert)
movie['keywords'] = movie['keywords'].apply(convert)
movie['cast'] = movie['cast'].apply(convert3)
movie['crew'] = movie['crew'].apply(fetch_director)
movie['overview'] = movie['overview'].apply(lambda x: x.split() if isinstance(x, str) else [])

In [14]:
for feature in ['genres', 'keywords', 'cast', 'crew']:
    movie[feature] = movie[feature].apply(lambda x: [i.replace(" ", "") for i in x])

In [55]:
movie['tags'] = movie['overview'] + movie['keywords'] + movie['genres'] + movie['cast'] + movie['crew']
new_df = movie[['movie_id', 'title', 'genres', 'tags']]
new_df['genres'] = new_df['genres'].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
new_df['genres'] = new_df['genres'].apply(lambda x: " ".join(x).lower())
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x).lower())

In [56]:
new_df.head()

,movie_id,title,genres,tags
0,19995,Avatar,action adventure fantasy sciencefiction,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,action adventure crime,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,action crime drama thriller,following the death of district attorney harve...
4,49529,John Carter,action adventure sciencefiction,"john carter is a war-weary, former military ca..."


# TFIDF Vectorization

In [57]:
tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
tfidf_matrix = tf.fit_transform(new_df['tags'])
similarity = cosine_similarity(tfidf_matrix)

In [58]:
similarity

array([[1.        , 0.02740263, 0.01660637, ..., 0.0198464 , 0.01297098,
        0.00678894],
       [0.02740263, 1.        , 0.03023385, ..., 0.02691934, 0.01909833,
        0.01118466],
       [0.01660637, 0.03023385, 1.        , ..., 0.02806676, 0.01282505,
        0.01218114],
       ...,
       [0.0198464 , 0.02691934, 0.02806676, ..., 1.        , 0.04001028,
        0.0286156 ],
       [0.01297098, 0.01909833, 0.01282505, ..., 0.04001028, 1.        ,
        0.03653644],
       [0.00678894, 0.01118466, 0.01218114, ..., 0.0286156 , 0.03653644,
        1.        ]])

# Chatbot intent classification


In [60]:
intents = json.load(open("intents.json"))
words, classes, documents = [], [], []
ignore_words = ['?', '!', ',', '.', ':', ';', '(', ')', '[', ']']

In [61]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [62]:
words = sorted(set([lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]))
classes = sorted(set(classes))

In [63]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [64]:
training, output_empty = [], [0] * len(classes)
for doc in documents:
    bag = [1 if w in [lemmatizer.lemmatize(word.lower()) for word in doc[0]] else 0 for w in words]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [65]:
random.shuffle(training)
training = np.array(training, dtype=object)
train_x, train_y = list(training[:, 0]), list(training[:, 1])

# Building the intent classification model

In [66]:
model = Sequential([
    Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(train_y[0]), activation='softmax')
])

In [67]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [68]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 128)                 │           7,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,586 (64.79 KB)

 Trainable params: 16,586 (64.79 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0968 - loss: 2.2891      
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2310 - loss: 2.2248 
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2867 - loss: 2.1506 
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2024 - loss: 2.1882 
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2041 - loss: 2.1049     
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1517 - loss: 2.1257     
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2270 - loss: 1.9718 
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3989 - loss: 1.8579 
Epoch 9/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2515 - loss: 1.9540 
Epoch 10/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4532 - loss: 1.7348 
Epoch 11/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2324 - loss: 1.8453     
Epoch 12/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accur

In [70]:
model.save('chatbot_model.h5')

# Load the trained model for the chatbot


In [71]:
model = load_model('chatbot_model.h5')
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Movie recommendation function

In [59]:
# def recommend(movie_title):
#     movie_index = new_df[new_df['title'] == movie_title].index[0]
#     distances = similarity[movie_index]
#     movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
#     recommended_movies = [new_df.iloc[i[0]].title for i in movies_list]
#     return recommended_movies

In [120]:
# Function to recommend movies by title
def recommend_by_movie_title(movie_title):
    title = movie_title.lower()
    if title in new_df['title'].str.lower().values:
        movie_index = new_df[new_df['title'].str.lower() == title].index[0]
        distances = similarity[movie_index]
        movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
        recommended_movies = [new_df.iloc[i[0]].title for i in movies_list]
        return recommended_movies
    else:
        return ["Sorry, I couldn't find any movies with that title."]


In [121]:

# Function to recommend movies by genre
def recommend_by_genre(genre):
    genre = genre.lower()
    recommendations = new_df[new_df['genres'].apply(lambda x: genre in x.lower())]['title'].sample(5).tolist()
    return recommendations if recommendations else ["Sorry, I couldn't find any movies in that genre."]


In [114]:
# Preprocessing functions for user input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [115]:
def bag_of_words(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return np.array(bag)

In [116]:
def predict_class(sentence, model):
    bow = bag_of_words(sentence, words, show_details=False)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [122]:
# Function to check if the message contains a movie title
def extract_movie_title(message):
    # Simple approach: Check if a movie title exists in the dataset
    for title in new_df['title']:
        if title.lower() in message.lower():
            return title
    return None

# Function to get response from the intent

In [123]:
def get_response(intents_list, intents_json):
    if not intents_list:
        return "I'm sorry, I didn't understand that."

    tag = intents_list[0]['intent']
    # Check if the intent is movie title-based recommendation
    if tag == "movie_recommendation":
        movie_title = intents_list[0].get('movie_title', None)
        if movie_title:
            recommendations = recommend_by_movie_title(movie_title)
            if recommendations:
                return "Here are some movies similar to {}:\n{}".format(movie_title, "\n".join(recommendations))
            else:
                return "Sorry, I couldn't find any movies similar to {}.".format(movie_title)
        else:
            return "Please specify a movie title for recommendations."

    # Check if the intent is genre-based recommendation
    if tag == "genre_recommendation":
        genre = intents_list[0].get('genre', None)
        if genre:
            recommendations = recommend_by_genre(genre)
            if recommendations:
                return "Here are some {} movies you might enjoy:\n{}".format(genre, "\n".join(recommendations))
            else:
                return "Sorry, I couldn't find any {} movies.".format(genre)
        else:
            return "Please specify a genre (e.g., action, comedy, horror)."

    # Default response for other intents
    list_of_intents = intents_json['intents']
    for intent in list_of_intents:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])

    return "I'm sorry, I didn't understand that."

In [124]:
# Function to identify genre in user message
def extract_genre(message):
    genres = ["action", "comedy", "drama", "horror", "sci-fi", "romantic", "thriller"]  # Add genres as needed
    for genre in genres:
        if genre in message.lower():
            return genre
    return None


# Chatbot conversation loop

In [125]:
print("Bot is running!")

# Main conversation loop
while True:
    message = input("User >> ")
    if message == "quit":
        break

    # First, check if the user message contains a genre
    genre = extract_genre(message)
    if genre:
        # If a genre is detected, create a genre-based intent
        intents_list = [{'intent': 'genre_recommendation', 'genre': genre}]
    else:
        # If no genre is detected, try to extract a movie title
        movie_title = extract_movie_title(message)
        if movie_title:
            # If a movie title is found, create a movie recommendation intent
            intents_list = [{'intent': 'movie_recommendation', 'movie_title': movie_title}]
        else:
            # Otherwise, classify the intent using your model
            intents_list = predict_class(message, model)

    if intents_list:
        response = get_response(intents_list, intents)
    else:
        response = "I'm sorry, I didn't understand that. Could you please rephrase?"

    print("Cruella_bot >>", response)

Bot is running!
User >> hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Cruella_bot >> Hey! What can I do for you?
User >> suggest movie like Avata
Cruella_bot >> Here are some movies similar to O:
Semi-Pro
Romeo + Juliet
The Curse of Downers Grove
Juwanna Mann
About Last Night
User >> suggest action movie
Cruella_bot >> Here are some action movies you might enjoy:
The Punisher
Nothing to Lose
The Yards
Terminator 2: Judgment Day
Batman: The Dark Knight Returns, Part 2
User >> Batman
Cruella_bot >> Here are some movies similar to Batman:
Batman
Batman & Robin
Batman Returns
The Dark Knight Rises
The Dark Knight
User >> drama movie
Cruella_bot >> Here are some drama movies you might enjoy:
August Rush
The Exploding Girl
Gangster Squad
Julie & Julia
Beyond the Valley of the Dolls
User >> thanks
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Cruella_bot >> You're welcome!
User >> bye
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Cruella_bot >> See you soon!
User >> quit
